In [1]:
# Import packages needed for analysis of data
import pandas as pd
import plotly.express as px
import re
from matplotlib import pyplot as plt 
import numpy as np 
from scipy import stats 
import seaborn as sns
import streamlit as st 

In [2]:
# Read in the data set
df = pd.read_csv("vehicles_us.csv")
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51525 entries, 0 to 51524
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         51525 non-null  int64  
 1   model_year    47906 non-null  float64
 2   model         51525 non-null  object 
 3   condition     51525 non-null  object 
 4   cylinders     46265 non-null  float64
 5   fuel          51525 non-null  object 
 6   odometer      43633 non-null  float64
 7   transmission  51525 non-null  object 
 8   type          51525 non-null  object 
 9   paint_color   42258 non-null  object 
 10  is_4wd        25572 non-null  float64
 11  date_posted   51525 non-null  object 
 12  days_listed   51525 non-null  int64  
dtypes: float64(4), int64(2), object(7)
memory usage: 5.1+ MB
None


In [3]:
# Get an example of the DataFrame, print the first and last 5 rows
print(df.head())
print(df.tail())

   price  model_year           model  condition  cylinders fuel  odometer  \
0   9400      2011.0          bmw x5       good        6.0  gas  145000.0   
1  25500         NaN      ford f-150       good        6.0  gas   88705.0   
2   5500      2013.0  hyundai sonata   like new        4.0  gas  110000.0   
3   1500      2003.0      ford f-150       fair        8.0  gas       NaN   
4  14900      2017.0    chrysler 200  excellent        4.0  gas   80903.0   

  transmission    type paint_color  is_4wd date_posted  days_listed  
0    automatic     SUV         NaN     1.0  2018-06-23           19  
1    automatic  pickup       white     1.0  2018-10-19           50  
2    automatic   sedan         red     NaN  2019-02-07           79  
3    automatic  pickup         NaN     NaN  2019-03-22            9  
4    automatic   sedan       black     NaN  2019-04-02           28  
       price  model_year           model  condition  cylinders fuel  odometer  \
51520   9249      2013.0   nissan ma

In [ ]:
# Check for missing values
print(df.isna().sum())

price               0
model_year       3619
model               0
condition           0
cylinders        5260
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          25953
date_posted         0
days_listed         0
dtype: int64


In [ ]:
# Check for duplicates in model
print(df['model'].duplicated().sum())

51425


In [ ]:
# See if we can't fill in any information missing in the 'cylinders' column by using the duplicates grouped by the 'model' and 'model_year' column
df['cylinders'] = (
    df.groupby(['model', 'model_year'])['cylinders']
      .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))
)

In [ ]:
# Review how many missing values are now in the cylinders column
print(df['cylinders'].isna().sum())

3645


By using information from duplicates that were grouped by the columns 'model' and 'model_year' we were able to decrease missing values for the 'cylinders' column from a total of 5,260 to 3,645, a difference of 1,615. This will help provide for better calculations and overall data collection.

In [21]:
# By using the newly filled data, we will try and fill in missing data for the column 'is_4wd'
df['is_4wd'] = (
    df.groupby(['model', 'model_year'])['is_4wd']
      .transform(lambda x: x.fillna(x.mode()[0] if not x.mode().empty else x))
)
print(df['is_4wd'].isna().sum())

15496


By using the columns 'model' and 'model_year' we were able to decrease the number of missing values from 25,953 to 15,496; a difference of 10,457.

In [17]:
# Double check calculations are correct and the entire df reflects findings from above
print(df.isna().sum())


price               0
model_year       3619
model               0
condition           0
cylinders        3645
fuel                0
odometer         7892
transmission        0
type                0
paint_color      9267
is_4wd          15496
date_posted         0
days_listed         0
dtype: int64


There is no reliable way to fill in missing values for the model_year column, we could find the median, or even possibly the average, based off of the information from the other filled in columns, but this will not be accurate and could scew data one way or another when doing a deeper dive into it.

There is no way to fill in the information for the columns 'odometer' and 'paint_color' as these are specific to each individual car, when doing exploratory analysis of the data moving forward it will need to be noted for any calculations that may involve these columns.

In [24]:
print(df.describe())

               price    model_year    cylinders       odometer   is_4wd  \
count   51525.000000  47906.000000  47880.00000   43633.000000  36029.0   
mean    12132.464920   2009.750470      6.11982  115553.461738      1.0   
std     10040.803015      6.282065      1.65655   65094.611341      0.0   
min         1.000000   1908.000000      3.00000       0.000000      1.0   
25%      5000.000000   2006.000000      4.00000   70000.000000      1.0   
50%      9000.000000   2011.000000      6.00000  113000.000000      1.0   
75%     16839.000000   2014.000000      8.00000  155000.000000      1.0   
max    375000.000000   2019.000000     12.00000  990000.000000      1.0   

       days_listed  
count  51525.00000  
mean      39.55476  
std       28.20427  
min        0.00000  
25%       19.00000  
50%       33.00000  
75%       53.00000  
max      271.00000  
